In [1]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

# Importação

In [2]:
# Carrega a base de dados contendo os registros de importação de fertilizantes
im=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/i.csv').drop(columns='date')

# Elimina ano incompleto
im=im[im.CO_ANO!=2024]

im=im[im.KG_LIQUIDO>0]

# Modifica unidade de peso
im.KG_LIQUIDO=im.KG_LIQUIDO/10**6
im=im.rename(columns={'KG_LIQUIDO':'kton'})
im.head(1)

,CO_ANO,CO_MES,kton
0,1997,3,7.505776


# Chuvas

In [3]:
ch=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/c.csv').drop(columns='date')
ch=ch[ch.CO_ANO!=2001]
ch.tail(1)

,CO_ANO,CO_MES,mm
264,2023,12,78.6


Modelo Preditivo  

Random Forest Regressor  

Variável Target: Previsão de importação de fertilizantes em kton/mês

Variáveis independentes:
* Testemunha: Ano e mês
* Tratamento: Ano, mês e volume de chuvas

Hipóteses sobre o tratamento e a testemunha: 

* Ho    = "O tratamento não tem efeito, manténdo o mesmo o erro médio absoluto entre testemunha e tratamento"

* H_pos = "O tratamento tem efeito positivo, reduzindo o erro médio absoluto em relação a testemnha"
    * 

* H_neg = "O tratamento tem efeito negativo, aumentando o erro médio absoluto em relação a testemnha"

Executa os testes

In [4]:
q=pd.DataFrame()
#q=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q.csv')

from sklearn.ensemble import RandomForestRegressor as RFR
# Define hiperparametro
md=1
# Instância o regresssor
RFR=RFR(max_depth=md, random_state=1)

i=im.copy()
c=ch.copy()

# Determina que o adiantamento dos meses em relação as importações seja igual ao adinatamento de máximo efeito, 
# como determinado no capítulo anterior
ad=7

# Adianta os meses
c.CO_MES=c.CO_MES+ad

# Corriges meses fora da faixa
for adi in range(1,ad+1):
    c.loc[c.CO_MES==12+adi,'CO_ANO']=c.loc[c.CO_MES==12+adi]['CO_ANO']+1
    c.loc[c.CO_MES==12+adi,'CO_MES']=adi

# Funde chuvas com importações
i=i.merge(c)

In [5]:
i[i.kton==0]

,CO_ANO,CO_MES,kton,mm


In [6]:
i=i[i.kton!=0]

In [7]:
# Recria coluna de data
i['date']=pd.to_datetime(i.CO_ANO.astype(str)+'/'+i.CO_MES.astype(str)+'/1').astype(int)

# Descarta coluna Ano, substituida por data
i=i.drop(columns='CO_ANO')

# Formata mês como informação categórica
i.CO_MES=i.CO_MES.astype('category')

i=pd.get_dummies(i)

for a in range(1,len(i)):


    for train_size_mes in [10*12]:


        train=i.iloc[:a,:].tail(train_size_mes)
        
        if len(train)>=train_size_mes:


            
            test=i.iloc[a:a+1,:]
                        
            # Train
            X_train_tratamento=train.drop(columns='kton')
            X_train_testemunha=train.drop(columns=['kton','mm'])
            y_train=train.kton
    
            # Test
            X_test_tratamento  =test.drop(columns='kton')
            X_test_testemunha=test.drop(columns=['kton','mm'])
            y_test=test.kton
                
            RFR.fit(X_train_tratamento,y_train)
            
            test_pred=test     .assign(kton_pred_tratamento  =RFR.predict(X_test_tratamento  ).astype(int))
            
            RFR.fit(X_train_testemunha,y_train)
            
            test_pred=test_pred.assign(kton_pred_testemunha=RFR.predict(X_test_testemunha).astype(int))

            test_pred=test_pred.assign(ad=ad,train_size_anos=train_size_mes/12,md=md)
            
            q=pd.concat([q,test_pred])

q['erro_tratamento']=np.abs(q.kton-q.kton_pred_tratamento)
q['erro_testemunha']=np.abs(q.kton-q.kton_pred_testemunha)

print('erro_tratamento.mean',round(q.erro_tratamento.mean()))
print('erro_testemunha.mean',round(q.erro_testemunha.mean()))

erro_tratamento.mean 39
erro_testemunha.mean 44


Salva o resultado dos testes

In [8]:
q.to_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q.csv', index=False)